# Setup Env

In [1]:
!pip install fastai --upgrade -q
from fastai.vision.all import *
# from fastai.callback.cutmix import *

!pip install wwf -q
!pip install timm -q
from wwf.vision.timm import *

# !pip install efficientnet_pytorch -q
# from efficientnet_pytorch import EfficientNet

# Import data

In [2]:
path = Path('../input/planets-dataset/planet/planet')

In [3]:
train_df = pd.read_csv(path/'train_classes.csv')
train_df

In [4]:
def get_x(r):
    return path/'train-jpg'/(r['image_name']+'.jpg')

def get_y(r):
    return r['tags'].split()

def get_data(size=224,bs=64,data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                       splitter=RandomSplitter(seed=42),
                       get_x=get_x, 
                       get_y=get_y,
                       item_tfms = Resize(size),
                       batch_tfms = [*aug_transforms(flip_vert=True,max_warp=0),
                                     Normalize.from_stats(*imagenet_stats)]
                      )
    return dblock.dataloaders(data_df,bs=bs)

In [5]:
dls = get_data(300,40)

In [6]:
dls.show_batch(nrows=1, ncols=3)

# Training

In [7]:
f2samples = FBetaMulti(beta=2,average='samples',thresh=0.2)


metrics = [partial(accuracy_multi, thresh=0.2), f2samples]
cbs = [MixUp]

In [8]:
learn = timm_learner(dls, 'efficientnet_b3', metrics=metrics, cbs=cbs)


In [9]:
learn.fine_tune(12, base_lr=3e-2, freeze_epochs=4)

# Submission using TTA

In [12]:
file_path = Path('../input/planets-dataset/test-jpg-additional/test-jpg-additional')
test_path = Path('../input/planets-dataset/planet/planet/test-jpg')
submission_df = pd.read_csv(path/'sample_submission.csv')
testing_path = (submission_df['image_name'] + '.jpg').apply(lambda x: test_path/x if x.startswith('test') else file_path/x)

def prediction(filename='submission.csv', tta=False):
    tst_dl = learn.dls.test_dl(testing_path)
    if tta:
        predictions = learn.tta(dl = tst_dl)
    else:
        predictions = learn.get_preds(dl = tst_dl)
    predlist = [' '.join(learn.dls.vocab[i]) for i in (predictions[0] > 0.2)]

    df = submission_df
    df['tags'] = predlist

    df.to_csv(filename, index=False)
    return df

In [13]:
prediction('submission_tta.csv', tta=True)